## Part-4.4：Chain -> LLMRouterChain、MultiPromptChain：实现条件判断的大模型调用

In [ ]:
import os
import openai
import configparser
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import DatetimeOutputParser
from langchain import LLMChain
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain


conf = configparser.ConfigParser()
current_directory = os.path.dirname(os.path.realpath('__file__'))
config_file_path = os.path.join(current_directory, '..', '..', '..', 'config.ini')
conf.read(config_file_path)
api_key = conf.get("Openai", "api_key")  # 在config.ini中配置自己的APIkey
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")
chat_model = "gpt-3.5-turbo"
text_model = "text-davinci-003"

In [ ]:
llm = OpenAI(openai_api_key=api_key)

In [ ]:
from langchain import ConversationChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

templates = [
    {
        "name": "游戏设计师（Game Designer）",
        "description": "适合回答设计游戏的玩法、关卡、角色和系统，确保游戏的乐趣和挑战性。",
        "template": "你是游戏设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }, {
        "name": "角色设计师（Character Designer）",
        "description": "适合回答创造游戏中的各种角色，包括主角、配角、敌人、NPC（非玩家角色）等",
        "template": "你是角色设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }, {
        "name": "关卡设计师（Level Designer）",
        "description": "适合回答设计游戏中的各个关卡、场景和环境",
        "template": "你是关卡设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }, {
        "name": "美术设计师（Art Designer）",
        "description": "适合回答游戏的视觉风格、角色造型、场景设计等美术方面的工作。",
        "template": "你是美术设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }
]

destination_chains = {}
for tem in templates:
    name = tem['name']
    description = tem["description"]
    template = tem["template"]
    prompt = PromptTemplate(template=template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key="text")

In [ ]:
from langchain.chains.router.llm_router import RouterOutputParser, LLMRouterChain

destinations = [f"{p['name']}: {p['description']}" for p in templates]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()

)

router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True)

In [ ]:
from langchain.chains import MultiPromptChain

mult_chain = MultiPromptChain(
    router_chain=router_chain,
    default_chain=default_chain,
    destination_chains=destination_chains,
    verbose=True
)

In [ ]:
print(mult_chain.run({"input": "设计一个6岁的ncp小男孩的特征"}))

In [ ]:
result = mult_chain.run({'input': '设计一个宇宙空间站的游戏场景'})
